#### Error Handling

pydantic will raise `ValidationError` whenever it finds an error in the data it's validating.

> ##### Note
> 
> Validation code should not raise `ValidationError` itself, but rather raise `ValueError`, `TypeError` or `AssertionError` (or subclasses of `ValueError` or `TypeError`) which will be caught and used to populate `ValidationError`.

One exception will be raised regardless of the number of errors found, that `ValidationError` will contain information about all the errors and how they happened. You can access these errors in several ways:

* `e.errors()`: method will return list of errors found in the input data.

* `e.json()`: method will return a JSON representation of `errors`.

* `str(e)`: method will return a human readable representation of the errors.

Each error object contains:

* `loc`: the error's location as a list. The first item in the list will be the field where the error occurred, and if the field is a `sub-model`, subsequent items will be present to indicate the nested location of the error.

* `type`: a computer-readable identifier of the error type.

* `msg`: a human readable explanation of the error.

* `ctx`: an optional object which contains values required to render the error message.

In [1]:
from typing import List
from pydantic import BaseModel, ValidationError, PydanticValueError, conint, validator

In [2]:
class Location(BaseModel):
    lat = 0.1
    lng = 10.1

In [3]:
class Model(BaseModel):
    is_required: float
    gt_int: conint(gt=42)
    list_of_ints: List[int] = None
    a_float: float = None
    recursive_model: Location = None

In [4]:
data = dict(
    list_of_ints=["1", 2, "bad"],
    a_float="not a float",
    recursive_model={"lat": 4.2, "lng": "New York"},
    gt_int=21,
)

In [5]:
try:
    print(Model(**data))
except ValidationError as e:
    print(e)

5 validation errors for Model
is_required
  field required (type=value_error.missing)
gt_int
  ensure this value is greater than 42 (type=value_error.number.not_gt; limit_value=42)
list_of_ints -> 2
  value is not a valid integer (type=type_error.integer)
a_float
  value is not a valid float (type=type_error.float)
recursive_model -> lng
  value is not a valid float (type=type_error.float)


In [6]:
try:
    print(Model(**data))
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "is_required"
    ],
    "msg": "field required",
    "type": "value_error.missing"
  },
  {
    "loc": [
      "gt_int"
    ],
    "msg": "ensure this value is greater than 42",
    "type": "value_error.number.not_gt",
    "ctx": {
      "limit_value": 42
    }
  },
  {
    "loc": [
      "list_of_ints",
      2
    ],
    "msg": "value is not a valid integer",
    "type": "type_error.integer"
  },
  {
    "loc": [
      "a_float"
    ],
    "msg": "value is not a valid float",
    "type": "type_error.float"
  },
  {
    "loc": [
      "recursive_model",
      "lng"
    ],
    "msg": "value is not a valid float",
    "type": "type_error.float"
  }
]


##### Custom Errors

In your custom data types or validators you should use `ValueError`, `TypeError` or `AssertionError` to raise errors.

See `validators` for more details on use of the `@validator` decorator.

In [7]:
class Model(BaseModel):
    foo: str

    @validator("foo")
    def value_must_equal_bar(cls, v):
        if v != "bar":
            raise ValueError("value must be 'bar'")

        return v

In [8]:
try:
    print(Model(foo="ber"))
except ValidationError as e:
    print(e.errors())

[{'loc': ('foo',), 'msg': "value must be 'bar'", 'type': 'value_error'}]


You can also define your own error classes, which can specify a custom error code, message template, and context.

In [9]:
class NotABarError(PydanticValueError):
    code = "not_a_bar"
    msg_template = "value is not 'bar', got '{wrong_value}'"

In [10]:
class FooModel(BaseModel):
    foo: str

    @validator("foo")
    def value_must_equal_bar(cls, v):
        if v != "bar":
            raise NotABarError(wrong_value=v)
        return v

In [11]:
try:
    print(Model(foo='ber'))
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "foo"
    ],
    "msg": "value must be 'bar'",
    "type": "value_error"
  }
]
